In [1]:
import numpy as np
from scipy.linalg import expm
import numpy.linalg as LA
import random

from qiskit import QuantumCircuit
from qiskit import Aer, transpile,execute
from qiskit.quantum_info import random_pauli, state_fidelity, diamond_norm, Choi
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.tools.visualization import plot_histogram, plot_state_city

def create_hamiltonian(Nq, cn, gamma, Bn):
    '''
    create Hamiltonian gate
    target Hamiltonian is H=1/2*cn((1+gamma)XX+(1-gamma)YY)+BnZn
    Nq -> number of qubit
    cn -> coupling constant
    gamma -> parameter
    Bn -> Magnetic field
    '''
    
    XX= np.array(np.zeros(2**Nq))
    YY= np.array(np.zeros(2**Nq))
    Zn= np.array(np.zeros(2**Nq))
    Identity = 'I' * Nq
    hamiltonian = np.array(np.zeros(2**Nq))
    
    for k in range(0, Nq):
        '''
        隣あうqubitのみ相互作用するようなモデルを考える
        0 1 2 ... k-1 X X k+2 ... N-1
        0 1 2 ... k-1 Y Y k+2 ... N-1
        '''
        
        # 端まで計算したらスキップ
        if k == Nq-1:
            continue

        hamiX = Pauli(Identity[:k] + 'XX' + Identity[k+2:])
        hamiY = Pauli(Identity[:k] + 'YY' + Identity[k+2:])

        XX = XX + 0.5*cn[k]*(1+gamma)*hamiX.to_matrix()
        YY = YY + 0.5*cn[k]*(1-gamma)*hamiY.to_matrix()

    for k in range(0, Nq):
        hamiZ = Pauli(Identity[:k] + 'Z' + Identity[k+1:])
        Zn = Zn + Bn[k] * hamiZ.to_matrix()
    
    return XX + YY + Zn
        

In [2]:
def create_choi(t_list, q, cn, r, bn, direct=True):
    '''
    loop -> loop count
    q -> number of qubit
    cn -> coupling constant
    r -> gamma parameter
    bn -> magnetic field
    direct -> direct or indirect control
    '''
    
    qc = QuantumCircuit(q)
    random.shuffle(t_list)
    for t in t_list:
        # Hamiltonianの時間発展を計算
        hami = expm(-1j*create_hamiltonian(q,cn,r,bn)*t)
        qc.append(Operator(hami),list(range(q)))

#         各bitに独立な確率でPauliゲートを追加
        if direct:
            qc.append(random_pauli(1), [random.randint(0,q-1)])
        else:
            qc.append(random_pauli(1), [random.randint(0,1)])

    return Choi(qc).data

In [7]:
import datetime

def run(t_direct, t_indirect):
    qubit = 5
    cn = [1] * qubit #[1,1,1,1,1]
    r = 0
    bn = [0] * qubit #[0,0,0,0,0]

    choi_qubit = 2 * qubit
    choi_direct = np.array(np.zeros(2**choi_qubit))
    choi_indirect = np.array(np.zeros(2**choi_qubit))

    # 100取ればオッケーなのか？
    # もっととって、収束するか確認する。平均値のばらつきがなくなるように
    # その上で、t_listを色々変えていく。
    # Pauliをやる比率とか
    loop = 100
    for i in range(0, loop):
        print(datetime.datetime.now())
        choi_direct = choi_direct + create_choi(t_direct, qubit, cn, r, bn, True)
        choi_indirect = choi_indirect + create_choi(t_indirect, qubit, cn, r, bn, False)

    eps = (choi_direct - choi_indirect) / loop
    print(diamond_norm(Choi(eps)))


In [8]:
# totalの時間は一定で、各tは決められた範囲でランダムにとる
# gate数60,　time: 0.005 ~ 0.5 (sum: 10)
t_direct = [0.005, 0.005, 0.005, 0.005, 0.005, 0.015, 0.015, 0.015, 0.015, 0.015, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5]
# gate数100,　time: 0.01 ~ 0.4 (sum: 10)
t_indirect = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.3, 0.4, 0.4, 0.4]
run(t_direct, t_indirect)

2022-01-17 14:05:45.293939
2022-01-17 14:12:29.431463
2022-01-17 14:19:09.638916
2022-01-17 14:25:40.246035
2022-01-17 14:32:19.442308
2022-01-17 14:38:49.053252
2022-01-17 14:45:27.438023
2022-01-17 14:51:57.257631
2022-01-17 14:58:35.838018
2022-01-17 15:05:05.520868
2022-01-17 15:11:44.066466
2022-01-17 15:18:13.598795
2022-01-17 15:24:52.414693
2022-01-17 15:31:22.038544
2022-01-17 15:37:59.076426
2022-01-17 15:44:28.701068
2022-01-17 15:51:04.693736
2022-01-17 15:57:34.606056
2022-01-17 16:04:10.870845
2022-01-17 16:10:40.431606
2022-01-17 16:17:17.281562
2022-01-17 16:23:46.839020
2022-01-17 16:30:24.717310
2022-01-17 16:36:54.097463
2022-01-17 16:43:30.259154
2022-01-17 16:49:59.814284
2022-01-17 16:56:41.309152
2022-01-17 17:03:16.707081
2022-01-17 17:09:54.933481
2022-01-17 17:16:23.435386
2022-01-17 17:23:01.081667
2022-01-17 17:29:29.600145
2022-01-17 17:36:06.443863
2022-01-17 17:42:35.060388
2022-01-17 17:49:12.307456
2022-01-17 17:55:41.222057
2022-01-17 18:02:17.335984
2

KeyboardInterrupt: 

In [20]:
# totalの時間は一定で、各tは決められた範囲でランダムにとる
# gate数20,　time: 0.005 ~ 2 (sum: 10)
t_direct = [0.005, 0.005, 0.05, 0.05, 0.05, 0.05, 0.1, 0.2, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.99, 1, 2, 2]
# gate数100,　time: 0.01 ~ 0.4 (sum: 10)
t_indirect = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.3, 0.4, 0.4, 0.4]
run(t_direct, t_indirect)

2022-01-14 01:54:59.974572
2022-01-14 01:59:54.751325
2022-01-14 02:04:43.302009
2022-01-14 02:09:31.701258
2022-01-14 02:14:22.830060
2022-01-14 02:19:08.678482
2022-01-14 02:23:53.731220
2022-01-14 02:28:41.282018
2022-01-14 02:33:25.048775
2022-01-14 02:38:10.672512
2022-01-14 02:42:58.337441
2022-01-14 02:47:42.161633
2022-01-14 02:52:26.984750
2022-01-14 02:57:14.422858
2022-01-14 03:01:58.743095
2022-01-14 03:06:42.839874
2022-01-14 03:11:29.719635
2022-01-14 03:16:12.068707
2022-01-14 03:20:56.272880
2022-01-14 03:25:43.797667
2022-01-14 03:30:26.652257
2022-01-14 03:35:11.026121
2022-01-14 03:39:57.259222
2022-01-14 03:44:41.209814
2022-01-14 03:50:02.211425
2022-01-14 03:55:18.439164
2022-01-14 04:00:01.921006
2022-01-14 04:04:45.765754
2022-01-14 04:09:34.782965
2022-01-14 04:14:18.749659
2022-01-14 04:19:02.536151
2022-01-14 04:23:49.662335
2022-01-14 04:28:31.834322
2022-01-14 04:33:16.179700
2022-01-14 04:38:02.581907
2022-01-14 04:42:45.189360
2022-01-14 04:47:28.914541
2

In [14]:
# totalの時間は一定で、各tは決められた範囲でランダムにとる
# gate数40,　time: 0.005 ~ 2 (sum: 10)
t_direct = [0.005, 0.005, 0.005, 0.005, 0.025, 0.025, 0.025, 0.025, 0.05, 0.05, 0.05, 0.075, 0.075, 0.075, 0.075, 0.08, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.75, 1, 1, 1, 1]
# gate数100,　time: 0.01 ~ 0.4 (sum: 10)
t_indirect = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.3, 0.4, 0.4, 0.4]
run(t_direct, t_indirect)

2022-01-13 22:35:04.963918
2022-01-13 22:40:54.209415
2022-01-13 22:46:30.658858
2022-01-13 22:52:57.147545
2022-01-13 22:59:38.152996
2022-01-13 23:05:17.003761
2022-01-13 23:11:46.519937
2022-01-13 23:18:31.266726
2022-01-13 23:24:12.775796
2022-01-13 23:29:52.272558
2.0000244102915947


In [15]:
# totalの時間は一定で、各tは決められた範囲でランダムにとる
# gate数100,　time: 0.01 ~ 0.4 (sum: 10)
t_direct = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.3, 0.4, 0.4, 0.4]
# gate数100,　time: 0.01 ~ 0.4 (sum: 10)
t_indirect = [0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.25, 0.3, 0.4, 0.4, 0.4]
run(t_direct, t_indirect)

2022-01-14 00:01:29.448651
2022-01-14 00:09:34.535385
2022-01-14 00:17:34.554545
2022-01-14 00:25:37.311124
2022-01-14 00:33:36.471970
2022-01-14 00:41:32.722866
2022-01-14 00:49:36.661328
2022-01-14 00:57:43.998205
2022-01-14 01:05:47.825944
2022-01-14 01:13:51.849516
2.000033083843717
